In [1]:
%load_ext autoreload
%autoreload 2

from typet5.utils import proj_root, os
os.chdir(proj_root())

In [2]:
from typet5.experiments.utils import remove_newer_syntax_for_repo, Path
from typet5.experiments.typilus import eval_typilus_on_repos, TypilusSupportedSyntax
from typet5.utils import get_dataset_dir
import shutil

# dataset_name = "InferTypes4Py"
dataset_name = "ManyTypes4Py"
repos_dir = get_dataset_dir(dataset_name) / "repos"
shutil.rmtree(repos_dir / "test-typilus", ignore_errors=True)
shutil.copytree(repos_dir / "test", repos_dir / "test-typilus")
remove_newer_syntax_for_repo(repos_dir / "test-typilus", TypilusSupportedSyntax)


Removing newer syntax: 100%|██████████| 1594/1594 [00:01<00:00, 799.31it/s]

1594 / 1594 files have been rewritten.


In [5]:
from typet5.model import ModelWrapper
from typet5.static_analysis import AccuracyMetric
from typet5.utils import *
from typet5.experiments.typet5 import accs_as_table_row

test_repos = [p for p in (repos_dir / "test-typilus").iterdir() if p.is_dir()]

common_names = ModelWrapper.load_common_type_names(
    get_model_dir() / "model-v7--TrainingConfig(drop_env_types=False)"
)
metrics = AccuracyMetric.default_metrics(common_names)
typilus_path = Path("~/Projects/typilus-action/").expanduser()
work_dir = Path("~/Projects/typilus-action/data_out").expanduser()

cache = PickleCache(Path(f"caches/run_typilus"))
cache.remove(f"{dataset_name}.pkl")
accs = cache.cached(
    f"{dataset_name}.pkl",
    lambda: eval_typilus_on_repos(
        test_repos, metrics, typilus_path, work_dir, max_workers=4
    ),
)

accs_as_table_row(accs)
pretty_print_dict(accs)


Running Typilus: 100%|██████████| 50/50 [00:46<00:00,  1.09it/s]


Accuracies on all types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
47.10 & 54.05 & 55.12 & 33.23 & 60.37
Accuracies on common types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
47.10 & 54.05 & 55.12 & 33.23 & 60.37
Accuracies on rare types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
nan & nan & N/A & N/A & nan
full_acc:
   full_acc: 47.10% (count=6.9k)
   full_acc_by_cat:
      FuncArg: 47.86% (count=4.4k)
      FuncReturn: 46.79% (count=1.3k)
      ClassAtribute: 45.40% (count=1.1k)
      GlobalVar: 20.45% (count=44)
   full_acc_by_simple:
      complex: 20.42% (count=529)
      simple: 49.31% (count=6.4k)
   full_acc_label_size: 1.4992
   full_acc_pred_size: 1.1339
   full_acc_ignored_labels: 0
   n_missing: 3958
   n_skipped_rare: 4892
full_acc_common:
   full_acc_common: 47.10% (count=6.9k)
   full_acc_common_by_cat:
      FuncA